In [ ]:
# --- 1. 환경 설정 ---
#!pip install langchain langgraph faiss-cpu tiktoken python-dotenv

import os
import re
from dotenv import load_dotenv
from langchain.schema import HumanMessage, AIMessage, Document
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState  # ✅ 변경됨

# --- 2. API Key 불러오기 ---
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("API 키 확인:", OPENAI_API_KEY[:10] + "..." if OPENAI_API_KEY else "없음")

# --- 3. 메뉴 데이터 불러오기 ---
MENU_FILE = "../data/restaurant_menu.txt"
with open(MENU_FILE, "r", encoding="utf-8") as f:
    raw_text = f.read()

# 메뉴를 항목별로 분리
menu_blocks = re.split(r"\n\d+\. ", raw_text.strip())
menu_data = []
for block in menu_blocks:
    if not block.strip():
        continue
    lines = block.strip().split("\n")
    menu_name = lines[0].strip()
    content = "\n".join(lines[1:])
    menu_data.append({"menu_name": menu_name, "content": content})

# --- 4. 벡터 DB 구축 ---
embedding = OpenAIEmbeddings()
menu_db = FAISS.from_texts(
    [d["content"] for d in menu_data],
    embedding,
    metadatas=[{"menu_name": d["menu_name"]} for d in menu_data]
)

# --- 5. MessagesState 확장 ---
class CafeState(MessagesState):
    query_type: str | None = None
    search_query: str | None = None

# --- 6. 정보 추출 함수 ---
def extract_menu_info(doc: Document) -> dict:
    """검색된 문서에서 구조화된 메뉴 정보 추출"""
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    
    price_match = re.search(r"₩[\d,]+", content)
    desc_match = re.search(r"설명[:：]\s*(.+)", content, re.DOTALL)
    
    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": desc_match.group(1).strip() if desc_match else "설명 없음"
    }

def classify_query(state: CafeState) -> CafeState:
    user_message = state["messages"][-1].content  # .messages → ["messages"]
    
    if "가격" in user_message:
        state["query_type"] = "price"
        state["search_query"] = user_message.replace("가격", "").strip()
    elif "추천" in user_message or "뭐가 좋아" in user_message:
        state["query_type"] = "recommend"
        state["search_query"] = user_message
    else:
        state["query_type"] = "menu"
        state["search_query"] = user_message

    return state

def generate_response(state: CafeState) -> CafeState:
    query_type = state["query_type"]
    search_query = state["search_query"]

    if query_type == "price":
        docs = menu_db.similarity_search(search_query, k=3)
    elif query_type == "recommend":
        docs = menu_db.similarity_search(search_query, k=2)
        if not docs:
            docs = menu_db.similarity_search("인기 메뉴", k=2)
    else:
        docs = menu_db.similarity_search(search_query, k=3)

    infos = [extract_menu_info(d) for d in docs]

    if query_type == "price":
        response = "\n".join([f"{i['name']} → {i['price']}" for i in infos])
    else:
        response = "\n".join([f"{i['name']} ({i['price']}): {i['description']}" for i in infos])

    state["messages"].append(AIMessage(content=response))  # .messages → ["messages"]
    return state


# --- 9. 그래프 구성 ---
graph = StateGraph(CafeState)
graph.add_node("classify", classify_query)
graph.add_node("respond", generate_response)

graph.add_edge(START, "classify")
graph.add_edge("classify", "respond")
graph.add_edge("respond", END)

app = graph.compile()

# --- 10. 실행 예시 ---
init_state = CafeState(messages=[HumanMessage(content="스테이크 가격 알려줘")])
final_state = app.invoke(init_state)

# dict 접근
for m in final_state["messages"]:
    role = "사용자" if isinstance(m, HumanMessage) else "AI"
    print(f"{role}: {m.content}")



API 키 확인: sk-proj-at...
사용자: 스테이크 가격 알려줘
AI: 해당 메뉴의 가격 정보를 찾을 수 없습니다.
